# Extractice QA :  SQuAD

This is a simple notebook to download and store the SQuAD extractive qa dataset. We will take it from Huggingface datasets libray (<3) and turn it into one of the two formats that classy is able to parse (i.e. jsonl or tsv).

After creating this dataset you can train the model by exectuing the following bash command
```bash
classy train qa data/qa/squad -n my_firt_squad_run
```

In [19]:
! pip install datasets

In [20]:
from datasets import load_dataset
import os
from tqdm.notebook import tqdm

In [21]:
# here we load the dataset dataset from "datasets"
dataset = load_dataset("squad")

Reusing dataset squad (/home/edobobo/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [22]:
# let's create a repositiory that will contains the dataset splits
import os
dir_path = "squad"
os.mkdir(dir_path)
! ls

FileExistsError: [Errno 17] File exists: 'squad'

In [ ]:
# if you want the output format to be tab separated decomment the second line
output_format = "jsonl"
# output_format = "tsv"

if output_format == "jsonl":
    import json

In [32]:
from collections import Counter


for k in ['train', 'validation']:

    with open(f'{dir_path}/{k}.{output_format}', 'w') as f:

        for instance in tqdm(dataset[k]):

            title = instance["title"].replace("_", " ")
            context = instance["context"]
            title_add = f"{title.strip('.')}. "
            context = title_add + context

            question = instance["question"]
            answers_start = instance["answers"]["answer_start"]
            answers_text = instance["answers"]["text"]

            # we select the most voted answer from the annotators
            answer_text = max(Counter(answers_text).items(), key=lambda x: x[1])[0]
            answer_start = answers_start[answers_text.index(answer_text)] + len(title_add)
            answer_end = answer_start + len(answer_text)

            if output_format == "jsonl":
                json_dict = dict(
                    context=context,
                    question=question,
                    answer_start=answer_start,
                    answer_end=answer_end
                )
                dump_line = json.dumps(json_dict)
            else:
                dump_line = f"{context}\t{question}\t{answer_start}\t{answer_end}"

            f.write(dump_line)
            f.write("\n")

  0%|          | 0/87599 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

In [33]:
! head -5 $dir_path/train.$output_format

{"context": "University of Notre Dame. Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.", "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?", "answer_start": 541, "answer_end": 567}
{"context": "University of Notre Dame. Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a g

In [34]:
! head -5 $dir_path/validation.$output_format

{"context": "Super Bowl 50. Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50.", "question": "Which NFL team represented the AFC at Super Bowl 50?", "answer_start": 192, "answer_end": 206}
{"context": "Super Bowl 50. Super Bowl 50 was an American football game to de